In [93]:
import streamlit as st
import pandas as pd
import numpy as np
import pydeck as pdk

In [94]:
DATA_PATH= './data/full_col.csv'

def load_data(nrows=None):
    data = pd.read_csv(DATA_PATH, nrows=nrows)
    data[['lon', 'lat']] = data['coords'].str.split(',', expand=True).astype(float)
    return data


In [95]:
df = load_data(5000);
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ticket_id      5000 non-null   object 
 1   type           5000 non-null   object 
 2   organization   5000 non-null   object 
 3   comment        5000 non-null   object 
 4   photo          5000 non-null   object 
 5   photo_after    3485 non-null   object 
 6   coords         5000 non-null   object 
 7   address        5000 non-null   object 
 8   subdistrict    4998 non-null   object 
 9   district       4998 non-null   object 
 10  province       5000 non-null   object 
 11  timestamp      5000 non-null   object 
 12  state          5000 non-null   object 
 13  star           2299 non-null   float64
 14  count_reopen   5000 non-null   int64  
 15  last_activity  5000 non-null   object 
 16  type_str       5000 non-null   object 
 17  lon            5000 non-null   float64
 18  lat     

In [96]:
df['state'].unique()

array(['เสร็จสิ้น', 'กำลังดำเนินการ', 'รอรับเรื่อง'], dtype=object)

In [97]:
import ast
def parse_set_string(s):
    try:
        parsed = ast.literal_eval(s)
        return list(parsed) if isinstance(parsed, set) else []
    except:
        return []

In [98]:
import ast
df['type_list'] = df['type'].apply(lambda x: parse_set_string(x) if pd.notnull(x) else [])
# df = df.explode('type')
df['organization_list'] = df['organization'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
# df = df.explode('organization')
df.head()

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,...,timestamp,state,star,count_reopen,last_activity,type_str,lon,lat,type_list,organization_list
0,2021-FYJTFP,{'ความสะอาด'},['เขตบางซื่อ'],ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,...,2021-09-03 12:51:09.453003+00,เสร็จสิ้น,NaN,0,2022-06-04 15:34:14.609206+00,ความสะอาด,100.53084,13.81865,[ความสะอาด],[เขตบางซื่อ]
1,2021-CGPMUN,"{'ร้องเรียน', 'น้ำท่วม'}","['เขตประเวศ', 'ฝ่ายโยธา เขตประเวศ']",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.66709,13.67891",189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,...,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,0,2022-06-21 08:21:09.532782+00,"น้ำท่วม,ร้องเรียน",100.66709,13.67891,"[ร้องเรียน, น้ำท่วม]","[เขตประเวศ, ฝ่ายโยธา เขตประเวศ]"
2,2021-7XATFA,{'สะพาน'},['เขตสาทร'],สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,"100.52649,13.72060",191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,...,2021-09-26 05:03:52.594898+00,เสร็จสิ้น,NaN,0,2022-06-06 01:17:12.272904+00,สะพาน,100.52649,13.72060,[สะพาน],[เขตสาทร]
3,2021-9U2NJT,{'น้ำท่วม'},"['เขตบางซื่อ', 'ฝ่ายโยธา เขตบางซื่อ']",น้ำท่วม,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.53099,13.81853",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,...,2021-10-14 10:45:27.713884+00,เสร็จสิ้น,NaN,0,2022-09-08 08:35:43.784519+00,น้ำท่วม,100.53099,13.81853,[น้ำท่วม],"[เขตบางซื่อ, ฝ่ายโยธา เขตบางซื่อ]"
4,2021-DVEWYM,"{'น้ำท่วม', 'ถนน'}","['เขตลาดพร้าว', 'ฝ่ายโยธา เขตลาดพร้าว']",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.59165,13.82280",702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,...,2021-12-09 12:29:08.408763+00,เสร็จสิ้น,5.0,0,2022-08-12 07:18:44.884945+00,"น้ำท่วม,ถนน",100.59165,13.82280,"[ถนน, น้ำท่วม]","[เขตลาดพร้าว, ฝ่ายโยธา เขตลาดพร้าว]"


In [99]:
df['state'] = df['state'].apply(lambda x: x if x else "Unknown")
state_unique = df['state'].unique()
color_map = {
    "เสร็จสิ้น" : [0,255,0],
    "กำลังดำเนินการ" : [0,0,255],
    "รอรับเรื่อง" : [255,0,0],
    "Unknown" : [0,0,0]
}
# print(color_map)
df['state_color'] = df['state'].apply(lambda state: color_map[state])

In [100]:
df['state_color']

0       [0, 255, 0]
1       [0, 255, 0]
2       [0, 255, 0]
3       [0, 255, 0]
4       [0, 255, 0]
           ...     
4995    [0, 0, 255]
4996    [0, 255, 0]
4997    [0, 255, 0]
4998    [0, 255, 0]
4999    [0, 255, 0]
Name: state_color, Length: 5000, dtype: object

In [108]:
df['district'].unique().tolist()

[nan,
 'ประเวศ',
 'สาทร',
 'ลาดพร้าว',
 'ดุสิต',
 'ลาดกระบัง',
 'วัฒนา',
 'คลองสาน',
 'ปทุมวัน',
 'ตลิ่งชัน',
 'ดอนเมือง',
 'ห้วยขวาง',
 'สะพานสูง',
 'สวนหลวง',
 'หลักสี่',
 'ธนบุรี',
 'บางนา',
 'ทุ่งครุ',
 'บางแค',
 'คลองสามวา',
 'จตุจักร',
 'บางเขน',
 'สายไหม',
 'บางซื่อ',
 'ภาษีเจริญ',
 'ดินแดง',
 'บางกอกน้อย',
 'วังทองหลาง',
 'ราษฎร์บูรณะ',
 'ราชเทวี',
 'บางขุนเทียน',
 'พระโขนง',
 'พระนคร',
 'บึงกุ่ม',
 'พญาไท',
 'จอมทอง',
 'มีนบุรี',
 'บางกอกใหญ่',
 'บางกะปิ',
 'บางคอแหลม',
 'หนองจอก',
 'ยานนาวา',
 'บางพลัด',
 'คลองเตย',
 'บางบอน',
 'คันนายาว',
 'หนองแขม',
 'สัมพันธวงศ์',
 'บางรัก',
 'ป้อมปราบศัตรูพ่าย',
 'บางกรวย',
 'ทวีวัฒนา',
 'พระประแดง']